### Sourmash vs bowtie2
Interesting: 
Sourmash predicts approx half the amount that read mapping does, every time. Same for RGI. 
Want to try this with MAGs, and compare sourmash manysearch to read mapping with bowtie2.

#### Steps:
- Sketch 5 metags individually (R1, R2, merged) (have 3 already). Use sketch_individ_reads.smk
- Create a bowtie2 db for all 95% ANI MAGs
- Map reads to this (from 5 metaGs)
- Manysearch against a concatenated db for all reads. 



In [ ]:
# MAG locations
/group/ctbrowngrp2/scratch/annie/2024-pigparadigm/results/MAGs/drep.95/dereplicated_genomes/*.fasta

# metaGs:
/group/ctbrowngrp2/amhorst/2025-pigparadigm/results/sketches_metag

In [ ]:
srun --account=ctbrowngrp -p med2 -J testsketch -t 4:00:00 -c 20 --mem=100gb --pty bash


mamba activate branchwater-skipmer

snakemake -s sketch_individ_reads.smk --use-conda --resources mem_mb=100000 --rerun-triggers mtime \
-c 20 --rerun-incomplete -k -n

In [ ]:
# Build bowtie2 db from MAGs (use 95% ANi for now)
# concatenate all MAGs
cat /group/ctbrowngrp2/scratch/annie/2024-pigparadigm/results/MAGs/drep.95/dereplicated_genomes/*.fasta > MAGs.95.fa

# Build bowtie db 
srun --account=ctbrowngrp -p med2 -J bt2 -t 3:00:00 -c 32 --mem=70gb --pty bash
mamba activate bowtie2
bowtie2-build MAGs.95.fa MAGs_95 -p 32 --large-index 


# Map with the readmap.smk
snakemake -s readmap.smk --use-conda --resources mem_mb=100000 --rerun-triggers mtime \
-c 32 --rerun-incomplete -k -n

# Use coverM to get num reads mapped:
mamba activate coverm

# make a coverage table, where the min amount of the contig that has to be covered is 75%
coverm contig -m count \
-b *.bam > 250227_coverM_mag.tsv

coverm genome --genome-fasta-directory \
/group/ctbrowngrp2/scratch/annie/2024-pigparadigm/results/MAGs/drep.95/dereplicated_genomes/ \
-x fasta -m relative_abundance -b *.bam > 250227_coverM_bins.csv

In [ ]:
# Mags in sourmash format:
/group/ctbrowngrp2/scratch/annie/2024-pigparadigm/results/sourmash/sketches/MAGs.drep95.zip

# gotta rename some
sourmash sig rename SRR8960303_r1.zip "SRR8960303_r1" -o rn/SRR8960303_r1.zip
sourmash sig rename SRR8960303_r2.zip "SRR8960303_r2" -o rn/SRR8960303_r2.zip
sourmash sig rename SRR15057928_r2.zip "SRR15057928_r2" -o rn/SRR15057928_r2.zip
sourmash sig rename SRR14369135_r1.zip "SRR14369135_r1" -o rn/SRR14369135_r1.zip
sourmash sig rename SRR14369135_r2.zip "SRR14369135_r2" -o rn/SRR14369135_r2.zip

# manysearch x the individ reads
sourmash scripts manysearch \
/group/ctbrowngrp2/scratch/annie/2024-pigparadigm/results/sourmash/sketches/MAGs.drep95.zip \
fp.txt -o MAGs.k21s1000.R1R2.csv -k 21 -s 1000 -c 20


In [ ]:
# fastgather too?
for f in *_merge.zip
do 
sourmash scripts fastgather \
$f /group/ctbrowngrp2/scratch/annie/2024-pigparadigm/results/sourmash/sketches/MAGs.drep95.zip \
-k 21 --scaled 1000\
-c 32 -o fastgather/$f.csv
done